In [52]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

True

In [53]:
pc =Pinecone(api_key=os.getenv("PINECONE_DATABASE_SECRET_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,  
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '2df5e640c0cd645d6fc52bf200cf95ff', 'Date': 'Sat, 31 Aug 2024 12:56:02 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [54]:
import json
data =json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Mathematics',
  'stars': '5',
  'review': "Dr. Johnson's lectures are clear and engaging. She makes complex topics easy to understand."},
 {'professor': 'Dr. Bob Williams',
  'subject': 'Physics',
  'stars': '4',
  'review': 'Dr. Williams is knowledgeable and passionate, but his exams are tough.'},
 {'professor': 'Dr. Carol Smith',
  'subject': 'Chemistry',
  'stars': '3',
  'review': 'Dr. Smith is a decent professor, but her lectures can be a bit monotonous.'},
 {'professor': 'Dr. David Brown',
  'subject': 'Biology',
  'stars': '5',
  'review': 'Dr. Brown is an excellent professor who truly cares about his students.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Computer Science',
  'stars': '4',
  'review': 'Dr. Davis is very knowledgeable, but her assignments can be quite challenging.'},
 {'professor': 'Dr. Frank Miller',
  'subject': 'Economics',
  'stars': '2',
  'review': "Dr. Miller's lectures are difficult to follow, and he

In [63]:
processed_data = []
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client = OpenAI()
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["proffessor"],
        "metadata": {
        "review" :review["review"],
        "subject":review["subject"],
        "stars" : review["stars"]
        }
    })

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [64]:
processed_data[0]

IndexError: list index out of range

In [69]:

index= pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 31 Aug 2024 15:16:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '35', 'x-pinecone-request-id': '4325405545381095664', 'x-envoy-upstream-service-time': '36', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}
